In [1]:
#ライブラリを読み込む
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#複数ページから情報を取得する
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13109&sc=13110&sc=13112&sc=13114&sc=13115&sc=13116&sc=13117&cb=0.0&ct=20.0&et=5&md=07&md=10&cn=5&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page={}'

#変数d_listに空のリストを作成する
d_list = []

In [3]:
#1~2336をループする
for i in range(1,4):
    #変数target_urlに、アクセス先のURLを格納する
    target_url = url.format(i)
   
    #target_URLへのアクセス結果を変数rに格納
    r =requests.get(target_url)
    
    #サーバーに負荷をかけないために、１秒おきにfor文を実行する
    sleep(1)

    #取得結果を解析してsoupに格納
    soup = BeautifulSoup(r.text)

    #すべての物件情報を取得する
    contents = soup.find_all("div",class_="cassetteitem")

    #各物件情報をforループで取得する
    for content in contents:
       #物件情報・部屋情報を取得する
       detail = content.find("div",class_="cassetteitem-detail")
       table = content.find("table",class_="cassetteitem_other")
       #物件情報を取得する
       title = detail.find("div",class_="cassetteitem_content-title").text 
       address = detail.find("li",class_="cassetteitem_detail-col1").text
       accsess = detail.find("li",class_="cassetteitem_detail-col2").text
       age,floor = detail.find("li",class_="cassetteitem_detail-col3").text.split()
       #物件情報から各部屋の情報を取得する
       tr_tags = table.find_all("tr",class_="js-cassette_link")
       for tr_tag in tr_tags:
          #部屋情報の行から欲しい情報を取得する
          floor,price,first_fee,capacity = tr_tag.find_all("td")[2:6]
          #更に細かい情報を取得する
          fee,management_fee = price.find_all("li")
          deposit,gratuity = first_fee.find_all("li")
          madori,menseki = capacity.find_all("li")
          #取得した情報を辞書に格納する
          d = {
             "title":title,
             "address":address,
             "accsess":accsess,
             "age":age,
             "floor":floor.text,
             "fee":fee.text,
             "management_fee":management_fee.text,
             "deposit":deposit.text,
             "gratuity":gratuity.text,
             "madori":madori.text,
             "menseki":menseki.text
             }
          #取得した辞書をd_listに格納する
          d_list.append(d)


In [4]:
#変数d=listを使って、データフレームを作成する
df = pd.DataFrame(d_list)

In [5]:
#データクレンジングを実施する
# age列の処理: 築年数のみを抽出し、新築の場合は0に設定
import re
df['age'] = df['age'].apply(lambda x: 0 if '新築' in x else (int(re.search(r'築(\d+)年', x).group(1)) if re.search(r'築(\d+)年', x) else 0))

# floor列の処理: 階数の整数部分を抽出
def floor_number(i):
    if '-' in i:  # メゾネットタイプの場合
        return i.split('階')[0]
    elif 'B' in i:  # 地下の場合
        return int(re.search(r'B(\d+)', i).group(1)) * -1
    else:
        return int(re.search(r'(\d+)階', i).group(1))

df['floor'] = df['floor'].apply(floor_number)

# 金額関連の列(fee, deposit, gratuity)の処理: 万円を整数に変換
def yen_int(yen):
    if yen in ['-', '']:
        return 0
    search = re.search(r'([\d.]+)万?円', yen) 
    if search:
        return int(float(search.group(1)) * 10000)
    return 0  # 万円の形式に一致しない場合は0を返す

# management_fee 列の処理: 万円を整数に変換しない
def yen_int_no_conversion(yen):
    if yen in ['-', '']:
        return 0
    search = re.search(r'([\d.]+)万?円', yen) 
    if search:
        return int(float(search.group(1)))  # 万円を整数に変換せずそのまま返す
    return 0  # 万円の形式に一致しない場合は0を返す

df['fee'] = df['fee'].apply(yen_int)
df['management_fee'] = df['management_fee'].apply(yen_int_no_conversion)  # management_fee 列だけ別の処理を適用
df['deposit'] = df['deposit'].apply(yen_int)
df['gratuity'] = df['gratuity'].apply(yen_int)

# area列の処理: 数値のみを抽出
df['menseki_m2'] = df['menseki'].apply(lambda x: float(re.search(r'([\d.]+)m2', x).group(1)))
df = df.drop("menseki",axis=1)


In [6]:
len(df)

179

In [7]:
# データフレームから重複を削除
df_ = df.drop_duplicates(['title','address', 'accsess', 'age', 'floor', 'fee', 'management_fee', 'deposit', 'gratuity', 'madori', 'menseki_m2'])


In [8]:
len(df_)

172

In [15]:
# 必要なライブラリをインポート
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

# jsonファイルを使って認証情報を取得
SCOPES = [
    'https://www.googleapis.com/auth/drive',
    'https://spreadsheets.google.com/feeds'
]
SERVICE_ACCOUNT_FILE = 'suumo-scraiping-f6d1afe73c54.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, SCOPES)

# 認証情報を使ってスプレッドシートの操作権を取得
gs = gspread.authorize(credentials)

# 共有したスプレッドシートのキーを使ってシートの情報を取得
SPREADSHEET_KEY = '16DJ352ip2iwRoNppkMK_rMm5_YYlwEkdi_xiceSGSfA'
workbook = gs.open_by_key(SPREADSHEET_KEY)

# 新しいワークシートを作成
workbook.add_worksheet(title="suumo_scraping5", rows=200, cols=20)
set_with_dataframe(workbook.worksheet("suumo_scraping5"),df_,include_index=True)